In [18]:
import numpy as np
import pandas as pd
from spicy import stats
import os
from sklearn.neighbors import KNeighborsClassifier

%run ../datos/datos.ipynb # me importo dataset

Implementación distancia coseno

In [20]:
def distCoseno(A,X):
  # producto punto de matriz-matriz
  prod = A @ X.T  # (entrenamiento x muestras) * (testeo x muestras).T = (entrenamiento x testeo)
  
  # normas de las filas de A y X
  normaA = np.linalg.norm(A, axis=1).reshape(-1, 1)  # (entrenamiento, 1)
  normaX = np.linalg.norm(X, axis=1).reshape(1, -1)  # (1, testeo)
  
  # producto de normas
  prodNorm = normaA * normaX  # (entrenamiento, testeo)
  
  # distancia del coseno
  res = 1 - (prod / prodNorm)
  
  return res

Implementación KNN

In [ ]:
def knn(X_train, X_test, y_train, y_test, k, distancia='coseno'): 
    acertados = 0
    
    # todas las distancias entre X_train y x_test
    if distancia == 'coseno':
        distancias = distCoseno(X_train, X_test)  # (entrenamiento x testeo)
    elif distancia == 'euclidiana':
        distancias = distancia_euclidea(X_train, X_test)  # (entrenamiento x testeo)

    # Obtener los índices de los k vecinos más cercanos para cada muestra de prueba
    k_indices = np.argsort(distancias, axis=0)[:k]  # (k, testeo)
    
    # Obtener las etiquetas de los k vecinos más cercanos
    k_labels = y_train[k_indices]  # (k, testeo)
    
    # Calcular la moda (etiqueta más frecuente) a lo largo de los k vecinos para cada muestra de prueba
    moda_labels, _ = stats.mode(k_labels, axis=0)
    moda_labels = moda_labels.flatten()  # Convertir a vector unidimensional
    
    # Comparar las etiquetas predichas con las etiquetas reales
    acertados = (moda_labels == y_test.flatten()).astype(int)

    # Calcular la precisión
    res = np.mean(acertados)
    
    # KNN de sklearn para verificar, si se quiere descomentar
    # if distancia == 'coseno':
    #     neigh = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    # elif distancia == 'euclidiana':
    #     neigh = KNeighborsClassifier(n_neighbors=k)
    # neigh.fit(X_train, y_train.ravel())
    # score = neigh.score(X_test, y_test.ravel())
    
    # print(f"- Mi knn: {res}")
    # print(f"- KNeighborsClassifier: {score}")
    # print("-------------------------------")
    
    return res

Implementación K-Fold (4)

In [22]:
def k_fold_4(x_train, y_train, k, distancia='coseno'):
    porcentajes_particiones = np.zeros(4)
    n = len(x_train)
    alto = n // 4
    indices = np.arange(n)
    
    for i in range(4):
        test_indices = indices[i * alto: (i + 1) * alto]
        train_indices = np.concatenate((indices[:i * alto], indices[(i + 1) * alto:]))

        # corremos knn
        porcentajes_particiones[i] = knn(x_train[train_indices], x_train[test_indices], 
                                         y_train[train_indices], y_train[test_indices], k, distancia)

    # Calcular y devolver el promedio
    return np.mean(porcentajes_particiones)

Punto opcional: Comparar KNN con distancia euclídea vs distancia coseno, realizando la búsqueda de hiper-parámetros para cada distancia.

In [ ]:
def distancia_euclidea(A, X):
    # Cuadrado de las normas (la suma de los cuadrados de cada fila)
    normaA = np.linalg.norm(A, axis=1).reshape(-1, 1) 
    normaX = np.linalg.norm(X, axis=1).reshape(1, -1) 
    
    # Producto punto entre A y X
    prod = np.dot(A, X.T)  
    
    # Aplicando la fórmula: sqrt(x^2 + y^2 - 2xy)
    distancias = np.sqrt(normaA**2 + normaX**2 - 2 * prod)
    
    return distancias